<a href="https://colab.research.google.com/github/sudeshna0501/twitter-sentiment-analysis/blob/main/bert%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/My Drive/Tweets.csv')

In [ ]:
X= df['text']
Y= df['airline_sentiment']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import np_utils

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(df['airline_sentiment'])

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [ ]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

In [ ]:
X = X.apply(clean_doc)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(X, dummy_y,test_size=0.2)

In [ ]:
X_train_lstm = X_train

In [ ]:
def joining(ele):
 return " ".join(str(v) for v in ele)

In [ ]:
char_X=X_train_lstm.apply(joining)


In [ ]:
snt_emb = X_train_lstm.apply(joining)

In [ ]:
snt_emb


4689     SouthwestAir That sound hopeful Its thin devic...
3708     united please help trace luggage put without n...
7966     JetBlue im sorry Boarding chaos underway Pleas...
4820     SouthwestAir know everybody flight time flying...
9575     USAirways Discovered flight Cancelled Flightle...
                               ...                        
8067     JetBlue going BDL DCA flights yesterday today ...
74       VirginAmerica worried great ride new plane gre...
2865     united think board back plane whatstheholdup C...
8604     JetBlue names Angel seriously cool Truly FANTA...
10557        USAirways flight RUDE RUDE RUDE service Awful
Name: text, Length: 11712, dtype: object

In [ ]:
snt_emb = snt_emb.tolist()

In [ ]:
def split(word):
    return list(word)

In [ ]:
char_X=char_X.apply(split)

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate


In [ ]:
# fit a tokenizer
def create_tokenizer(lines):
 tokenizer = Tokenizer()
 tokenizer.fit_on_texts(lines)
 return tokenizer


# calculate the maximum document length
def max_length(lines):
 return max([len(s.split()) for s in lines])


# encode a list of lines
def encode_text(tokenizer, lines, length):
 # integer encode
 encoded = tokenizer.texts_to_sequences(lines)
 # pad encoded sequences
 padded = pad_sequences(encoded, maxlen=length, padding='post')
 return padded




#for calculating length
def joining(ele):
 return " ".join(str(v) for v in ele)


In [ ]:
#for cnn

# create tokenizer
tokenizer_cnn= create_tokenizer(X_train)
# calculate max document length
joinned_train_cnn=X_train.apply(joining)
length_cnn= max_length(joinned_train_cnn)
# calculate vocabulary size
vocab_size_cnn = len(tokenizer_cnn.word_index) + 1
print('Max document length: %d' % length_cnn)
print('Vocabulary size: %d' % vocab_size_cnn)
# encode data
trainX_cnn= encode_text(tokenizer_cnn, X_train, length_cnn)
print(trainX_cnn.shape)


Max document length: 23
Vocabulary size: 11564
(11712, 23)


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 4.4 MB 59.8 MB/s 
     |████████████████████████████████| 1.2 MB 64.4 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 69.5 MB/s 
     |████████████████████████████████| 6.6 MB 6.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=dd7936f9a67fdcb85ac4158c45302dda3f0c62fce887193f711ec173f18f9a78
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
sentence_embeddings = sbert_model.encode(snt_emb)

In [ ]:
trainX_bert=pd.DataFrame(sentence_embeddings)

In [ ]:
# define the model
def define_model(length_cnn, vocab_size_cnn):
 # channel 1
 inputs1 = Input(shape=(length_cnn,))
 embedding1 = Embedding(vocab_size_cnn, 128, input_length=length_cnn)(inputs1)
 lstm1= Bidirectional(LSTM(100))(embedding1)
 flat1 =(Dropout(0.5))(lstm1)


 input4 =  Input(shape=(768,))

 denseb1=Dense(300, activation='tanh')(input4)

 # merge  , flat2, flat3
 merged = concatenate([flat1, denseb1])
 # interpretation
 dense1 = Dense(300, activation='relu')(merged)
 dense2 = Dense(100, activation='relu')(dense1)
 dense3 = Dense(20, activation='relu')(dense2)
 dense4 = Dense(10, activation='relu')(dense3)
 outputs = Dense(3, activation='softmax')(dense4)

 model = Model(inputs=[inputs1, input4], outputs=outputs)
 # compile
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # summarize
 print(model.summary())
 plot_model(model, show_shapes=True, to_file='multichannel.png')
 return model

In [ ]:

# define model
model = define_model(length_cnn, vocab_size_cnn)
print(model.summary())

#trainX_lstm,trainX_cnn,trainX_cnn,

# fit model
model.fit([trainX_cnn,trainX_bert], y_train, epochs=5, batch_size=20)
# save the model
model.save('model.h5')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 23, 128)      1480192     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 200)          183200      ['embedding[0][0]']              
                                                                                                  
 input_2 (InputLayer)           [(None, 768)]        0           []                               
                                                                                              

In [ ]:
X_test_lstm=X_test

In [ ]:
def joining(ele):
 return " ".join(str(v) for v in ele)

In [ ]:
char_X_test=X_test_lstm.apply(joining)

In [ ]:
def split(word):
    return list(word)

In [ ]:
char_X_test=char_X_test.apply(split)

In [ ]:
snt_emb_test = X_test_lstm.apply(joining)

In [ ]:
snt_emb_test = snt_emb_test.tolist()

In [ ]:
testing_embeddings= sbert_model.encode(snt_emb_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model



# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

  #for calculating length
def joining(ele):
  return " ".join(str(v) for v in ele)




# create tokenizer
tokenizer_cnn= create_tokenizer(X_train)
# calculate max document length
joinned_train_cnn=X_train.apply(joining)
length_cnn= max_length(joinned_train_cnn)
# calculate vocabulary size
vocab_size_cnn = len(tokenizer_cnn.word_index) + 1
print('Max document length: %d' % length_cnn)
print('Vocabulary size: %d' % vocab_size_cnn)
# encode data
trainX = encode_text(tokenizer_cnn, X_train, length_cnn)
testX = encode_text(tokenizer_cnn, X_test, length_cnn)
print(trainX.shape)



#testLabels = list(test_data['sentiment'])
# load the model
model = load_model('model.h5')

# evaluate model on training dataset
loss, acc = model.evaluate([trainX_cnn, trainX_bert], y_train, verbose=0)
print('Train Accuracy: %f' % (acc*100))

#testX_lstm,testX,testX,
# evaluate model on test dataset dataset
loss, acc = model.evaluate([testX, testing_embeddings], y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Max document length: 23
Vocabulary size: 11564
(11712, 23)
Train Accuracy: 95.124656
Test Accuracy: 78.961748


In [ ]:
print(classification_report(y_test, testX, digits=3))

ValueError: ignored